In [ ]:
from dash import Dash, html, dcc, Input, Output
import plotly.graph_objs as go
import plotly.express as px
import dash_bootstrap_components as dbc
import pandas as pd

# Load your data
df = pd.read_csv('df')# add here your own data set

# Group by date and calculate balance
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')

# Calculate total income and expense
total_income = df[df['Amount (EUR)'] > 0]['Amount (EUR)'].sum()
total_expense = df[df['Amount (EUR)'] < 0]['Amount (EUR)'].sum()

# Initialize the app with Bootstrap theme
app = Dash(__name__, external_stylesheets=[dbc.themes.CYBORG])

# Define app layout
app.layout = dbc.Container([
    html.H1("Budget Buddy", className='mb-4', style={'textAlign': 'center', 'color': '#6A5ACD'}),

    dbc.Row([
        dbc.Col([
            dcc.Dropdown(
                id='dropdown',
                value='Account Balance Over Time',
                clearable=False,
                options=[
                    {'label': 'Account Balance Over Time', 'value': 'Account Balance Over Time'},
                    {'label': 'Income vs. Expense', 'value': 'Income vs. Expense'},
                    {'label': 'Monthly Spending Trends', 'value': 'Monthly Spending Trends'},
                    {'label': 'Spending by Category (Top Categories)', 'value': 'Spending by Category (Top Categories)'},
                    {'label': 'Top 10 Most Expensive Categories', 'value': 'Top 10 Most Expensive Categories'},
                    {'label': 'Top 6 Spending Categories', 'value': 'Top 6 Spending Categories'}
                ]
            )
        ], width=6, className='mb-4'),
        dbc.Col([
            dcc.DatePickerRange(
                id='date-picker-range',
                display_format='YYYY-MM-DD',
                start_date=df['Date'].min(),
                end_date=df['Date'].max(),
                end_date_placeholder_text="Select a date"
            )
        ], width=6, className='mb-4')
    ]),

    html.Div(id='chart-container')
], fluid=True, style={'backgroundColor': '#FFFFFF'})

# Create interactivity between dropdown component, date range selector, and chart
@app.callback(
    Output('chart-container', 'children'),
    Input('dropdown', 'value'),
    Input('date-picker-range', 'start_date'),
    Input('date-picker-range', 'end_date')
)
def update_chart(selected_chart, start_date, end_date):
    if start_date and end_date:
        start_date = pd.to_datetime(start_date)
        end_date = pd.to_datetime(end_date)
        filtered_df = df[(df['Date'] >= start_date) & (df['Date'] <= end_date)]
    else:
        filtered_df = df

    if selected_chart == 'Account Balance Over Time':
        # Line Chart for Account Balance Over Time
        balance_by_date = filtered_df.groupby('Date')['Amount (EUR)'].sum().cumsum()
        balance_fig = go.Figure()
        balance_fig.add_trace(go.Scatter(x=balance_by_date.index, y=balance_by_date.values, mode='lines', name='Balance', line=dict(color='#9400D3')))
        balance_fig.update_layout(
            title='Account Balance Over Time',
            xaxis_title='Date',
            yaxis_title='Balance',
            height=600,
            width=900,
            plot_bgcolor='#FFFFFF',
            paper_bgcolor='#FFFFFF',
            font=dict(color='#000000')
        )
        return dcc.Graph(figure=balance_fig)
    
    elif selected_chart == 'Income vs. Expense':
        # Income vs. Expense Bar Chart
        total_income_filtered = filtered_df[filtered_df['Amount (EUR)'] > 0]['Amount (EUR)'].sum()
        total_expense_filtered = filtered_df[filtered_df['Amount (EUR)'] < 0]['Amount (EUR)'].sum()
        income_expense_fig = go.Figure()
        income_expense_fig.add_trace(go.Bar(x=['Income'], y=[total_income_filtered], name='Income', marker_color='#4169E1'))
        income_expense_fig.add_trace(go.Bar(x=['Expense'], y=[-total_expense_filtered], name='Expense', marker_color='#9400D3'))
        income_expense_fig.update_layout(
            title='Income vs. Expense',
            xaxis_title='Category',
            yaxis_title='Amount (EUR)',
            height=600,
            width=900,
            plot_bgcolor='#FFFFFF',
            paper_bgcolor='#FFFFFF',
            font=dict(color='#000000')
        )
        return dcc.Graph(figure=income_expense_fig)
    
    elif selected_chart == 'Monthly Spending Trends':
        # Monthly Spending Trend
        monthly_spending_filtered = filtered_df[filtered_df['Amount (EUR)'] < 0].groupby(filtered_df['Date'].dt.to_period('M'))['Amount (EUR)'].sum()
        monthly_spending_fig = go.Figure()
        monthly_spending_fig.add_trace(go.Scatter(x=monthly_spending_filtered.index.to_timestamp(), y=monthly_spending_filtered.values, mode='lines', name='Monthly Spending', line=dict(color='#4169E1')))
        monthly_spending_fig.update_layout(
            title='Monthly Spending Trends',
            xaxis_title='Month',
            yaxis_title='Total Spent',
            height=600,
            width=900,
            plot_bgcolor='#FFFFFF',
            paper_bgcolor='#FFFFFF',
            font=dict(color='#000000')
        )
        return dcc.Graph(figure=monthly_spending_fig)

    
    elif selected_chart == 'Spending by Category (Top Categories)':
        # Bar Chart for Top 10 Spending Categories
        spending_by_category_filtered = filtered_df[filtered_df['Amount (EUR)'] < 0]['Category'].value_counts().head(10)
        top_ten_spending_fig = go.Figure()
        top_ten_spending_fig.add_trace(go.Bar(x=spending_by_category_filtered.index, y=spending_by_category_filtered.values, marker_color='#9400D3'))
        top_ten_spending_fig.update_layout(
            title='Spending by Category (Top Categories)',
            xaxis_title='Category',
            yaxis_title='Amount',
            height=600,
            width=900,
            plot_bgcolor='#FFFFFF',
            paper_bgcolor='#FFFFFF',
            font=dict(color='#000000')
        )
        return dcc.Graph(figure=top_ten_spending_fig)
    
    elif selected_chart == 'Top 10 Most Expensive Categories':
        # Bar Chart for Average Spending by Category
        average_spending_filtered = filtered_df[filtered_df['Amount (EUR)'] < 0].groupby('Category')['Amount (EUR)'].mean().sort_values(ascending=False).head(10)
        average_spending_fig = go.Figure()
        average_spending_fig.add_trace(go.Bar(x=average_spending_filtered.index, y=average_spending_filtered.values, marker_color='#4169E1'))
        average_spending_fig.update_layout(
            title='Top 10 Most Expensive Categories',
            xaxis_title='Category',
            yaxis_title='Average Spending (EUR)',
            height=600,
            width=900,
            plot_bgcolor='#FFFFFF',
            paper_bgcolor='#FFFFFF',
            font=dict(color='#000000')
        )
        return dcc.Graph(figure=average_spending_fig)
    
    elif selected_chart == 'Top 6 Spending Categories':
        # Pie Chart for Top 6 Spending Categories
        top_6_spending_categories_filtered = filtered_df[filtered_df['Amount (EUR)'] < 0]['Category'].value_counts().nlargest(6)
        top_6_spending_fig = px.pie(values=top_6_spending_categories_filtered.values, names=top_6_spending_categories_filtered.index, title='Top 6 Spending Categories', color_discrete_sequence=px.colors.sequential.Plasma)
        top_6_spending_fig.update_layout(
            height=600,
            width=900,
            plot_bgcolor='#FFFFFF',
            paper_bgcolor='#FFFFFF',
            font=dict(color='#000000')
        )
        return dcc.Graph(figure=top_6_spending_fig)

    return None

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True, port=8002, use_reloader=False)
